# Lab 2-07

Samantha Sánchez Tinoco

In [1]:
import numpy as np

In [3]:
# Calcular el radio espectral
def radio_espectral_gauss_seidel(matriz_coeficientes):
    n = len(matriz_coeficientes)
    
    # Inicializar D, L y U
    D = [[0.0 if i != j else matriz_coeficientes[i][i] for j in range(n)] for i in range(n)]
    L = [[0.0 if j >= i else matriz_coeficientes[i][j] for j in range(n)] for i in range(n)]
    U = [[0.0 if j <= i else matriz_coeficientes[i][j] for j in range(n)] for i in range(n)]
    
    # Matriz de iteración de Gauss-Seidel: (D - L)^-1 * U
    DL_inv = np.linalg.inv([[D[i][i] - L[i][j] for j in range(n)] for i in range(n)])
    matriz_iteracion = [[sum(DL_inv[i][k] * U[k][j] for k in range(n)) for j in range(n)] for i in range(n)]
    
    # Calcular los valores propios de la matriz de iteración
    autovalores = np.linalg.eigvals(matriz_iteracion)
    
    # Radio espectral: valor absoluto del mayor autovalor
    radio_espectral = max(abs(val) for val in autovalores)
    
    return radio_espectral

In [6]:
def srs(matriz_coeficientes, vector_independientes, tolerancia=1e-6):

    # Verificar si el sistema converge usando el radio espectral
    radio_espectral = radio_espectral_gauss_seidel(matriz_coeficientes)
    if radio_espectral >= 1:
        return "El radio espectral es mayor o igual a 1, el sistema no converge", 0

    # Calcular el valor de omega usando la fórmula
    omega = 2 / (1 + (1 - radio_espectral**2)**0.5)

    # Continuar si converge
    n = len(vector_independientes)
    x = [0.0 for _ in range(n)]  # Inicializar las variables en cero
    contador_iteraciones = 0

    while True:
        x_nuevo = x[:]  # Copiar los valores actuales
        # Hacer los cálculos de las soluciones
        for i in range(n):
            suma_Ax = 0.0
            for j in range(n):
                if j != i:
                    suma_Ax += matriz_coeficientes[i][j] * x_nuevo[j]  # Usar el nuevo valor actualizado
            # Aplicar la relajación
            x_nuevo[i] = (1 - omega) * x[i] + omega * (vector_independientes[i] - suma_Ax) / matriz_coeficientes[i][i]

        # Calcular la norma infinita de la diferencia y la nueva tolerancia
        norma_infinito_diferencia = max(abs(x_nuevo[i] - x[i]) for i in range(n))
        norma_infinito_x_nuevo = max(abs(x_nuevo[i]) for i in range(n))

        tolerancia_actual = norma_infinito_diferencia / norma_infinito_x_nuevo  # Nueva tolerancia

        if tolerancia_actual < tolerancia: 
            return x_nuevo, contador_iteraciones  # Solución encontrada

        # Actualizar los valores y contar una iteración más
        x = x_nuevo[:]
        contador_iteraciones += 1


In [7]:
# Matriz ejemplo
matriz_coeficientes = [
    [5, 2, 1],
    [3, 6, 2],
    [1, 1, 4]
]

# Vector b
vector_independientes = [12, 27, 20]

# Llamar a la función 
solucion, iteraciones = srs(matriz_coeficientes, vector_independientes)

# Imprimir el resultado
print("Solución:", solucion)
print("Número de iteraciones:", iteraciones)

Solución: [np.float64(0.40229780962105754), np.float64(2.9080453514376767), np.float64(4.172414124884641)]
Número de iteraciones: 18
